# OFDM

0x1ACFFC1D is your friend.

Flag: flag{J9DGssH8Gc0vg5ZT}

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import sigmf

In [2]:
Nfft = 1024
Nused = 3 * 1024 // 4
skip = 1024 // 8
cyclic_pref = 1024 // 8

In [3]:
with open('ofdm.txt') as f_text:
    text = f_text.read()
text_bytes = np.frombuffer(bytes(text, encoding='ascii'), 'uint8')
text_syms = np.sum((1 - 2*np.unpackbits(text_bytes).reshape(-1, 2).astype('float'))
                    /np.sqrt(2) * np.array([1, 1j]),
                    axis=1)

In [4]:
syncword = np.unpackbits(np.array([0x1a, 0xcf, 0xfc, 0x1d], 'uint8'))
syncword_syms = np.sum((1 - 2*syncword.reshape(-1, 2).astype('float'))/np.sqrt(2) * np.array([1, 1j]),
                        axis=1)
syncword_symbol = np.tile(syncword_syms, Nused // syncword_syms.size)

In [5]:
symbols = np.zeros((160, Nfft), 'complex')
symbols[::8, skip:-skip] = syncword_symbol
data_sel = np.arange(symbols.shape[0]) % 8 != 0
symbols[data_sel, skip:-skip] = np.tile(text_syms, 3)[:symbols.shape[0]//8*7*Nused].reshape(-1, Nused)

In [6]:
x = np.fft.ifft(np.fft.fftshift(symbols, axes=1))
y = np.zeros((symbols.shape[0], Nfft + cyclic_pref), 'complex')
y[:, cyclic_pref:] = x
y[:, :cyclic_pref] = x[:, -cyclic_pref:]
y = y.ravel()

In [7]:
y.astype('complex64').tofile('ofdm.c64')

In [9]:
filename = 'ofdm101.sigmf-data'
# create the metadata
meta = sigmf.SigMFFile(
    data_file=filename, # extension is optional
    global_info = {
        sigmf.SigMFFile.DATATYPE_KEY: 'ci16_le',
        sigmf.SigMFFile.SAMPLE_RATE_KEY: 1920000,
        sigmf.SigMFFile.AUTHOR_KEY: 'Daniel Estevez <daniel@destevez.net>',
        sigmf.SigMFFile.DESCRIPTION_KEY: 'OFDM 101 (GRCon22 CTF)',
            sigmf.SigMFFile.VERSION_KEY: sigmf.__version__,
        }
    )

# create a capture key at time index 0
meta.add_capture(0, metadata={
    sigmf.SigMFFile.FREQUENCY_KEY: 1810750000,
    sigmf.SigMFFile.DATETIME_KEY: '2022-09-17T14:50:28.372Z',
})

# check for mistakes & write to disk
assert meta.validate()
meta.tofile(filename[:-4] + 'meta') # extension is optional